# Notebook for collection of data + EDA
# James Yu

In [1]:
import pandas as pd
from random import sample
import sys
import json
import time

In [3]:
#pd.read_csv('../data/dblp-v13/csv/2005.csv', encoding='unicode_escape', lineterminator='\n')
pd.read_csv('../data/dblp-v13/testcsv/2015.csv')

,Unnamed: 0,Title,Abstract,Keywords
0,0,Foreword.,Despite the global crisis which affects the wh...,miscellaneous
1,1,[Feedback],Presents readers' feedback and comments.,NaN
2,2,Foreword,The GREENS workshop provides a forum for pract...,NaN
3,3,Fahrerverhaltensmodelle,Die aktive Teilnahme am Straßenverkehr als Fah...,NaN
4,4,Fahrstreifenwechselassistenz,Fahrerassistenzsysteme dienen dazu für den Fa...,NaN
5,5,Diary.,Our roundup of what's on.,documentation miscellaneous general dental den...
6,6,Editorial,IEEE Transactions on Computer-Aided Design of ...,automated_driving road_safety hybrid_electric_...
7,7,Editorial.,This issue features a review of technological ...,Phytoplankton Macroalgae Nutrient_Data Daily_T...
8,8,Editorial.,The Journal of Applied Psychology is the oldes...,general documentation introductory_and_survey
9,9,Editorial.,The director of Cambodia's National AIDS Contr...,hiv_infections diseases developing_countries s...


## Looking at arXiv dataset from Kaggle

In [6]:
# arXiv dataset from Kaggle https://www.kaggle.com/neelshah18/arxivdataset
# fp1 = '../data/arxivData.json'
# df1 = pd.read_json(fp1)
# df1.value_counts('year')
# df1.value_counts('month')
# agg_titles_by_year = df1.groupby('year')['title'].apply(' '.join)
# agg_sum_by_year = df1.groupby('year')['summary'].apply(' '.join)
# agg_all_by_year = df1.groupby('year')['title'].apply(' '.join) + ' ' + df1.groupby('year')['summary'].apply(' '.join)

## Looking at DBLP v13 dataset

In [7]:
fp = '../data/dblpv13.json'

In [8]:
# DBLP v13 dataset
# file size is 16.1 GB
# Can try to read in as smaller pieces, or write out the data into .txt files by year

#df2 = pd.read_json(fp2)
#df2.head()

invalid_tags = ['"venue" :', '"_id" :', '"type" :', '"raw" :', '"raw_zh" :',
                    '"n_citation" :', '"page_start" :', '"page_end" :',
                    '"lang" :', '"volume" :', '"issue" :', '"issn" :',
                    '"isbn" :', '"doi" :', '"pdf" :', '"url" :', '"references" :']

In [9]:
# TODO: Create function that gathers value counts of papers by year
# 1. Get # of papers by year
# 2. Get # of papers by year for papers that have formatting issues
# 3. Get most common keywords
# 4. Get most common keywords by year

# TODO: Create a more lightweight function that doesn't store all of the content
#   (maybe just store info on length of title + abstract, year, number of keywords)

In [10]:
def print_stats(time, papers, issues, empty, year):
    print(str(time) + ' seconds has elapsed since start of function')
    print(str(papers) + ' papers processed')
    print(str(issues) + ' number of papers with json formatting issues')
    print(str(empty) + ' number of papers with empty abstracts')
    print(str(year) + ' number of papers with invalid/irrelevant years < 1950, > 2022')

In [11]:
def test(inpath):
    start = time.time()
    first_line = True
    tags = ['"title"', '"year"', '"keywords"', '"abstract"', '"fos"']
    valid_tag = True
    skip_paper = False
    num_papers, num_issues, num_abs, num_year = 0, 0, 0, 0

    content = '{'
    # Keeps track of the start of processing a block (individual paper)
    start_ind = False

    # TODO: Keeps track of # of papers by year
    counts = {}
    # TODO: Keeps track of # of papers by year with formatting issues
    issue_counts = {}

    with open(inpath, 'r', encoding='utf-8') as f:
        for line in f:
            # Indicates start of block
            if line == '{ \n':
                start_ind = True

            # Indicates end of block
            if line == '},\n':
                # Doesn't process the first paper (only contains info about structure)
                if first_line:
                    first_line = False
                    content = '{'
                    start_ind = False
                    continue
                # If there is an indicator to skip the paper
                if skip_paper:
                    content = '{'
                    skip_paper = False
                    start_ind = False
                    continue

                # Cleans up formatting of block
                content += '}\n'
                content = content.replace(' :', ':').replace(',', ', ')
                # Prevents issue with additional comma at end of dict
                if content[-4:-2] == ', ':
                    content = content[:-4] + '}\n'
                # Fix for issue with double beginning brackets
                if content[:2] == '{{':
                    content = content[1:]

                num_papers += 1
                try:
                    formatted = json.loads(content)
                    keywords = ' '.join([word.replace(' ', '_') for word in formatted['keywords']])
                    title = formatted['title'].replace(',', '').replace('\n', '').replace('\\', '')
                    abstract = formatted['abstract'].replace(',', '').replace('\n','').replace('\\', '')
                    if len(abstract) == 0:
                        num_abs += 1
                        continue
                    if formatted['year'] < 1950 or formatted['year'] > 2022:
                        num_year += 1
                        continue

                    if formatted['year'] not in counts:
                        counts[formatted['year']] = 0
                    counts[formatted['year']] += 1

                    # Writing out titles + abstracts to aggregated year .txt file
                    fp1 = '../data/dblp-v13/content_' + str(formatted['year']) + '.txt'
                    f1 = open(fp1, 'a')
                    f1.write(title + ' ' + abstract + ' ')
                    f1.close()
                    # Writing out DBLP keywords to aggregated year .txt file
                    fp2 = '../data/dblp-v13/keywords_' + str(formatted['year']) + '.txt'
                    f2 = open(fp2, 'a')
                    f2.write(keywords + ' ')
                    f2.close()
                except:
                    num_issues += 1

                    idx = content.find('"year":')
                    year = content[idx+8:idx+12]
                    try:
                        year = int(year)
                    except:
                        year = 'invalid'
                    if year not in issue_counts:
                        issue_counts[year] = 0
                    issue_counts[year] += 1

                # For testing purposes - printing info + stopping early
                # if num_papers == 100000:
                #     end = time.time()
                #     time_elapsed = end - start
                #     print_stats(time_elapsed, num_papers, num_issues, num_abs, num_year)
                #     return

                # Reset parameters
                start_ind = False
                content = '{'

            # Processes individual lines once we have found the start indicator
            if start_ind:
                line = line.strip()

                # We only care about title, abstract, keywords, year, fos tags
                if any([tag in line for tag in tags]):
                    valid_tag = True
                elif ' : ' in line:
                    # Catches invalid tags
                    valid_tag = False
                elif '.fr' in line:
                    # Had issues with french papers being included
                    skip_paper = True

                if not valid_tag:
                    continue

                if 'NumberInt' in line:
                    line = line.replace('NumberInt(', '')
                    line = line.strip()[:-2] + ','
                if line[-3:] == ': ,':
                    line = line.replace(': ,', ': null')
                if line == '"type" : 1,':
                    continue
                content += line

    end = time.time()
    time_elapsed = end - start
    #print_stats(time_elapsed, num_papers, num_issues, num_abs, num_year)
    return counts, issue_counts

In [12]:
out = test(fp)
out

({2011: 146868,
  2009: 142125,
  2001: 49350,
  2000: 46187,
  1966: 553,
  2012: 156854,
  2002: 59902,
  2004: 82101,
  1986: 6080,
  2010: 142752,
  2006: 112287,
  2013: 160375,
  1961: 252,
  2003: 67039,
  2007: 116154,
  1995: 24356,
  2005: 95545,
  1983: 4132,
  1979: 2973,
  1996: 27216,
  1988: 9362,
  2015: 169413,
  1977: 2505,
  1990: 12605,
  1992: 16528,
  1975: 2393,
  1989: 10601,
  1997: 30217,
  2008: 126724,
  1985: 5284,
  2014: 170577,
  1998: 34664,
  1962: 329,
  1993: 19292,
  1968: 928,
  1999: 39722,
  1984: 4431,
  1974: 2414,
  1981: 3596,
  1972: 1571,
  1987: 6976,
  1980: 3203,
  1982: 3861,
  1950: 2,
  1963: 335,
  1958: 168,
  1957: 151,
  1969: 942,
  1952: 52,
  1964: 403,
  1960: 212,
  1971: 1238,
  1991: 14574,
  1994: 22733,
  2017: 110687,
  1970: 1017,
  1953: 46,
  1976: 2522,
  2016: 104869,
  1959: 244,
  1973: 1902,
  1978: 2779,
  1967: 690,
  2018: 143727,
  1955: 75,
  1965: 415,
  2019: 133736,
  1956: 133,
  1951: 35,
  1954: 79,
  

In [13]:
import altair as alt
import matplotlib.pyplot as plt

In [62]:
actual = pd.DataFrame(pd.Series(out[0]).astype(int)).reset_index().rename(columns={'index': 'Year', 0: 'Count'})
actual

,Year,Count
0,2011,146868
1,2009,142125
2,2001,49350
3,2000,46187
4,1966,553
...,...,...
67,1956,133
68,1951,35
69,1954,79
70,2020,110836


In [76]:
errors = pd.DataFrame(pd.Series(out[1]).astype(int)).reset_index().rename(columns={'index': 'Year', 0: 'Count'})
errors

,Year,Count
0,1993,14033
1,2007,38942
2,2009,42113
3,2004,29079
4,1980,3103
...,...,...
148,1895,1
149,1877,1
150,1883,1
151,2021,47168


In [77]:
idx = 153
for year in range(1950, 2022):
    if year not in errors['Year'].values:
        errors.loc[idx] = [year, 0]
        idx += 1

In [78]:
alt.Chart(actual, title='# of papers per year').mark_bar().encode(x='Year', y='Count', tooltip=['Year', 'Count']).interactive()

alt.Chart(...)

In [79]:
alt.Chart(errors, title='# of papers (with json errors) per year').mark_bar().encode(x='Year', y='Count', tooltip=['Year', 'Count']).interactive()

alt.Chart(...)